In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
  Cloning https://github.com/huggingface/transformers.git (to revision main) to c:\users\administrator\appdata\local\temp\pip-req-build-vnzpyyod
  Resolved https://github.com/huggingface/transformers.git to commit d9dc993fdd6e6b0a61fe68ccbe838e00c73b9f80
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/388.6 kB ? eta -:--:--
     ---------------------------------- --- 348.2/388.6 kB 7.2 MB/s eta 0:00:01
     -------------------------------------- 388.6/388.6 kB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/269.5 kB ? eta -:--:--
     ------------------------------------  266.2/2

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\Administrator\AppData\Local\Temp\pip-req-build-vnzpyyod'

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install -q datasets accelerate

^C


In [29]:
from transformers import AutoProcessor, BlipForConditionalGeneration
import torch

processor = AutoProcessor.from_pretrained(r"E:\medical\depth\efficientteacher\runs/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(r"E:\medical\depth\efficientteacher\runs/blip-image-captioning-base")

In [22]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [33]:
import os 
import shutil
def split_from_csv(dataset_path):
    """
    make folder from csv files  
    :param dataset_path: 
    :return: 
    :param dataset_path: 
    :return: 
    """
  
    for file in ['train.csv', 'val.csv', 'test.csv']:
        file_path = os.path.join(dataset_path, file)
        with open(file_path) as f:
            lines = f.readlines()
            for line in lines[1:]:
                parts = line.split(",")
                if len(parts) != 2:
                    continue
                _dir = os.path.join(dataset_path, file[:-4])
                if not os.path.exists(_dir):
                    print("make _dir :", _dir)
                    os.makedirs(_dir)
                shutil.copy2(os.path.join(dataset_path, "images", parts[0]), _dir)
            shutil.copyfile(file_path, os.path.join(_dir, "metadata.csv"))

In [42]:
dataset_path = r"..\dataset\rhwu_v5"
split_from_csv(dataset_path)

In [38]:
from datasets import load_dataset

dataset = load_dataset("imagefolder",data_dir=os.path.join(dataset_path,"train"),split="train")

Resolving data files:   0%|          | 0/1584 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [39]:
dataset

Dataset({
    features: ['image', 'text'],
    num_rows: 1583
})

In [40]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)

In [41]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [31]:
model.train()

for epoch in range(50):
  loss_val=0
  print("Epoch:", epoch,",loss_val:",loss_val)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss
    loss_val = loss.item()
    print(loss_val)
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

Epoch: 0 ,loss_val: 0


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


13.083162307739258
10.288992881774902
Epoch: 1 ,loss_val: 0
10.21261978149414


KeyboardInterrupt: 